In [1]:
import numpy as np

from pydrake.all import *

import pandas as pd
from random import randrange

import manipulation
from manipulation.meshcat_cpp_utils import (
    StartMeshcat, MeshcatJointSliders)

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [4]:
def gripper_forward_kinematics_example():
    urdf_path = './urdftest1128mitmesh.urdf'
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    model = Parser(plant, scene_graph).AddModelFromFile(urdf_path, 'viz-robot')
    # Weld the robot to the world frame
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame())
    
    plant.Finalize()

    meshcat.Delete()
    meshcat.DeleteAddedControls()
    
    end_frame = "Link1"
    gripper_frame = plant.GetBodyByName(end_frame).body_frame()
#     AddMultibodyTriad(gripper_frame, scene_graph)

    meshcat.Delete()
    meshcat.DeleteAddedControls()

    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()

#     plant.SetPositions(plant.GetMyContextFromRoot(context),
#                   plant.GetModelInstanceByName("lcl-robot"),
#                   np.array([1.19000225, -0.5890859, -1.04350496, 0.]))

    gripper = plant.GetBodyByName(end_frame)
    def pose_callback(context):
        pose = plant.EvalBodyPoseInWorld(context, gripper)   ## This is the important line
        print(pose.translation())
        clear_output(wait=True)
        print("gripper position (m): " + np.array2string(
            pose.translation(), formatter={
                'float': lambda x: "{:3.2f}".format(x)}))
        print("gripper roll-pitch-yaw (rad):" + np.array2string(
            RollPitchYaw(pose.rotation()).vector(),
                         formatter={'float': lambda x: "{:3.2f}".format(x)}))
        print("pose rotation: ", pose.rotation())

    sliders = MeshcatJointSliders(meshcat, plant, context)
    sliders.Run(visualizer, context, pose_callback)

gripper_forward_kinematics_example()

RuntimeError: Error parsing file '/home/laniakea/git/research/DIVA-R1.1.3-V/src/urdftest1128mitmesh/meshes/base_link.obj' : Failed parse `f' line(e.g. zero value for face index. line 78.)
